In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

In [3]:
fhvhv_202011 = pd.read_parquet('../fhvhv_tripdata_2020-11.parquet', engine='pyarrow')
fhvhv_202011.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02884,B02884,2020-11-01 00:03:18,2020-11-01 00:06:29,2020-11-01 00:08:19,2020-11-01 00:16:49,157,82,1.98,...,0.90,0.00,NaN,0.0,6.62,N,N,,N,N
1,HV0003,B02884,B02884,2020-11-01 00:14:46,2020-11-01 00:19:57,2020-11-01 00:20:13,2020-11-01 00:30:35,82,7,3.86,...,1.29,0.00,NaN,0.0,12.64,N,N,,N,N
2,HV0003,B02884,B02884,2020-11-01 00:27:10,2020-11-01 00:36:01,2020-11-01 00:36:35,2020-11-01 00:56:51,7,36,7.57,...,2.56,0.00,NaN,0.0,24.70,N,N,,N,N
3,HV0003,B02884,B02884,2020-11-01 00:50:36,2020-11-01 00:59:17,2020-11-01 00:59:51,2020-11-01 01:04:41,36,36,0.90,...,0.93,0.00,NaN,0.0,13.56,N,N,,N,N
4,HV0003,B02682,B02682,2020-11-01 00:10:20,2020-11-01 00:16:13,2020-11-01 00:17:21,2020-11-01 00:30:23,249,232,2.07,...,2.26,2.75,NaN,9.5,25.81,N,N,,N,N


In [4]:
fhvhv_202011.shape

(11596865, 24)

In [5]:
fhvhv_202011.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11596865 entries, 0 to 11596864
Data columns (total 24 columns):
 #   Column                Non-Null Count     Dtype         
---  ------                --------------     -----         
 0   hvfhs_license_num     11596865 non-null  object        
 1   dispatching_base_num  11596865 non-null  object        
 2   originating_base_num  8367342 non-null   object        
 3   request_datetime      11596856 non-null  datetime64[ns]
 4   on_scene_datetime     8380962 non-null   datetime64[ns]
 5   pickup_datetime       11596865 non-null  datetime64[ns]
 6   dropoff_datetime      11596865 non-null  datetime64[ns]
 7   PULocationID          11596865 non-null  int64         
 8   DOLocationID          11596865 non-null  int64         
 9   trip_miles            11596865 non-null  float64       
 10  trip_time             11596865 non-null  int64         
 11  base_passenger_fare   11596865 non-null  float64       
 12  tolls                 1159

# 比較一般與共乘的基本費用

In [35]:
fhvhv_202011[fhvhv_202011.shared_match_flag == 'N'].base_passenger_fare.describe()

count    1.159576e+07
mean     1.817266e+01
std      1.360052e+01
min     -1.650300e+02
25%      9.320000e+00
50%      1.454000e+01
75%      2.254000e+01
max      1.431220e+03
Name: base_passenger_fare, dtype: float64

In [33]:
fhvhv_202011[fhvhv_202011.shared_match_flag == 'Y'].base_passenger_fare.describe()

count    1104.000000
mean        5.291866
std         3.406226
min         4.990000
25%         5.000000
50%         5.000000
75%         5.000000
max        95.950000
Name: base_passenger_fare, dtype: float64

# 計算等待時間

In [28]:
fhvhv_202011['rider_waiting_time'] = fhvhv_202011['on_scene_datetime'] - fhvhv_202011['request_datetime']
fhvhv_202011['rider_waiting_time'] = round(fhvhv_202011['rider_waiting_time'] / np.timedelta64(1, 's'), 0)


fhvhv_202011['driver_waiting_time'] = fhvhv_202011['pickup_datetime'] - fhvhv_202011['on_scene_datetime']
fhvhv_202011['driver_waiting_time'] = round(fhvhv_202011['driver_waiting_time'] / np.timedelta64(1, 's'), 0)

fhvhv_202011.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,rider_waiting_time,driver_waiting_time
0,HV0003,B02884,B02884,2020-11-01 00:03:18,2020-11-01 00:06:29,2020-11-01 00:08:19,2020-11-01 00:16:49,157,82,1.98,...,NaN,0.0,6.62,N,N,,N,N,191.0,110.0
1,HV0003,B02884,B02884,2020-11-01 00:14:46,2020-11-01 00:19:57,2020-11-01 00:20:13,2020-11-01 00:30:35,82,7,3.86,...,NaN,0.0,12.64,N,N,,N,N,311.0,16.0
2,HV0003,B02884,B02884,2020-11-01 00:27:10,2020-11-01 00:36:01,2020-11-01 00:36:35,2020-11-01 00:56:51,7,36,7.57,...,NaN,0.0,24.70,N,N,,N,N,531.0,34.0
3,HV0003,B02884,B02884,2020-11-01 00:50:36,2020-11-01 00:59:17,2020-11-01 00:59:51,2020-11-01 01:04:41,36,36,0.90,...,NaN,0.0,13.56,N,N,,N,N,521.0,34.0
4,HV0003,B02682,B02682,2020-11-01 00:10:20,2020-11-01 00:16:13,2020-11-01 00:17:21,2020-11-01 00:30:23,249,232,2.07,...,NaN,9.5,25.81,N,N,,N,N,353.0,68.0


In [29]:
fhvhv_202011.corr()

C:\Users\Ticy\AppData\Local\Temp\ipykernel_12364\2811895963.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  fhvhv_202011.corr()


,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,rider_waiting_time,driver_waiting_time
PULocationID,1.000000,0.130412,0.009554,-0.000260,0.015472,0.022142,0.020667,0.011793,0.171924,NaN,0.019870,0.009745,-0.021312,-0.005165
DOLocationID,0.130412,1.000000,0.089336,0.046657,0.092657,0.100524,0.102886,0.044539,0.146894,NaN,0.037155,0.082761,-0.014937,0.005021
trip_miles,0.009554,0.089336,1.000000,0.792141,0.888273,0.492764,0.850872,0.711710,0.065147,NaN,0.194051,0.921467,0.045090,0.062169
trip_time,-0.000260,0.046657,0.792141,1.000000,0.824026,0.376088,0.775797,0.730617,0.142035,NaN,0.186907,0.894787,0.021672,0.095472
base_passenger_fare,0.015472,0.092657,0.888273,0.824026,1.000000,0.467182,0.921096,0.780557,0.127333,NaN,0.234262,0.922019,0.034717,0.085574
tolls,0.022142,0.100524,0.492764,0.376088,0.467182,1.000000,0.597430,0.146251,-0.004124,NaN,0.132771,0.467413,0.013041,0.034750
bcf,0.020667,0.102886,0.850872,0.775797,0.921096,0.597430,1.000000,0.725403,0.117641,NaN,0.366054,0.884397,0.028308,0.072677
sales_tax,0.011793,0.044539,0.711710,0.730617,0.780557,0.146251,0.725403,1.000000,0.201386,NaN,0.183455,0.748984,0.034987,0.071661
congestion_surcharge,0.171924,0.146894,0.065147,0.142035,0.127333,-0.004124,0.117641,0.201386,1.000000,NaN,0.103223,0.109950,-0.103922,-0.004404
airport_fee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
